# Testing different SA methods 1/5
## nltk's naive bayes
This code was partly taken from [here](http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/)
TODO also attribute in thesis

In [18]:
# Imports
import nltk
import csv
import pickle
import random
import time

from gensim import corpora

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer(remove_stopwords=False)

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(tokenize(preprocess(text)), sentiment) for (topic, sentiment, id, text) in iterator if sentiment == "positive" or sentiment == "negative"]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)
    
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
train_tweets = tweets[:threshold]
test_tweets = tweets[threshold:]

4347


In [10]:
# TODO there might be a function like this in nltk?
def extract_features(document):
    document_words = set(document)
    features = {}
    # TODO do on dictionary without stopwords removed (results are the same though, see the sentiment analyzer notebook)
    for word in dictionary.token2id:
        features['contains(%s)' % word] = (word in document_words)
    return features

def test(classifier, test_tweets):
    labeled_correctly = 0
    for tweet in test_tweets:
        if classifier.classify(extract_features(tweet[0])) == tweet[1]:
            labeled_correctly+=1
    print("%d/%d = %.2d percent" % (labeled_correctly, len(test_tweets), labeled_correctly/len(test_tweets)*100))

Train and test the model with the dict from the streamed tweets and from the sanders dataset to show the descrepancies

In [11]:
# Use the dictionary we already created for the topic model
# With its own dict, the accuracy on the sanders data improves significantly
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_sanders.dict')
print("The dictionary has length %d" % len(dictionary.token2id))
t0 = time.time()
training_set = nltk.classify.apply_features(extract_features, train_tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Feature extraction and training took %d seconds" % (time.time() - t0))
test(classifier, test_tweets)

The dictionary has length 3961
Feature extraction and training took 9 seconds
69/95 = 72 percent


In [12]:
total = 0
labels = {"neutral":0,"irrelevant":0, "negative":0, "positive":0}
reality = {"neutral":0,"irrelevant":0, "negative":0, "positive":0}
for tweet in test_tweets:
    total+=1
    label = classifier.classify(extract_features(tweet[0]))
    labels[label]+=1
    reality[tweet[1]]+=1

In [13]:
for key in reality:
    label = labels[key]
    value = reality[key]
    print("%s & %d & %d\%% & %d & %d\%%" % (key, value, value/total*100, label,label/total*100))

neutral & 0 & 0\% & 0 & 0\%
irrelevant & 0 & 0\% & 0 & 0\%
negative & 47 & 49\% & 49 & 51\%
positive & 48 & 50\% & 46 & 48\%


In [14]:
# Try the same thing with the other dictionary
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_stream.dict')
t0 = time.time()
training_set = nltk.classify.apply_features(extract_features, train_tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)
"Feature extraction and training took %d seconds" % (time.time() - t0)
test(classifier, test_tweets)

70/95 = 73 percent


In [15]:
# Some showcasing
tweet = 'Larry is not my friend'
print(classifier.classify(extract_features(tokenize(preprocess(tweet)))))

positive


In [16]:
# Save the classifier if it's any good
save_classifier = open("./../../models/naive_bayes/nltk_naive_bayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

FileNotFoundError: [Errno 2] No such file or directory: './../../models/naive_bayes/nltk_naive_bayes.pickle'